# Scarica tweets partendo dai 'tweet_id' contenuti negli articoli di google news. Per ogni tweet estrai nuovi 'tweet_id' dalla sezione 'urls'.

##**Scarica le librerie necessarie**

In [ ]:
pip install tweepy

In [ ]:
pip install requests_html

In [ ]:
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

# Cartella da cui importare file = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING CLEAN TOM'   
# Cartella in cui esportare file post twitter = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON TWITTER TOM'  

# Spostati nella cartella contenente tutti i dataset di scraping
path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING CLEAN'
os.chdir(path)

# Importa tutti i file della cartella
# scrape_list = [i for i in glob.glob('*.json')]
scrape_list = [i for i in glob.glob('*.json')]

# Inseriscili in un dizionario
diz = {'{}'.format((m).split(' scraping')[0]): pd.read_json(m, orient='records') for m in scrape_list}

In [ ]:
# Per ogni film costruisci una lista contenente tutti i 'tweet_id'
dictio_list = {}
for n in diz:
  dictio_list[n] = []
  for row in diz[n]['twitter_id'].values:
    f = re.compile('(\d+)') #\d serve a prendere i numeri mentre il + le espressioni, metodo per compilare pattern di un espressione 
    for el in row:
        m = f.search(el)
        if m:
          k = m.group(1)
          if k not in dictio_list[n]: 
            dictio_list[n].append(int(k))
          else:
              dictio_list[n]

In [ ]:
# Controlla quanti tweet_id sono presenti in ogni dataset
for n in diz:
  print(len(diz[n]), len(dictio_list[n]), n)

##**Funzione API Twitter**

In [ ]:
import pandas as pd
import tweepy
import datetime
import re
def Prime(lista):
  # Importa le credenziali fornite da Twitter per utilizzare le API
  consumer_key = "uBJISqAHpRkCm79dKDwsBkHL1"
  consumer_secret = "NHvp1zx4Sk5n15iV4A1v2wK6Ygo7BAgheFOgEyRPmNcBkHthNq"
  access_key = "1460586759918600195-SR05QtXUgVfk7WjAJFe8RcG3a22XG0"
  access_secret = "MTwCZtmuHmvJqfxM943VqWq02GZvyeCj0Oflbit1w1y7n"
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_key, access_secret)  
  api = tweepy.API(auth, wait_on_rate_limit=True)

  def Post(lista):
      # Per superare il limite di ricerca di Twitter (pari a 100 tweets), prendi gruppi da 100 elementi alla volta dalla lista dei tweet_id.
      dictio = {}     
      # Per coprire tutti gli elementi nella lista imposta l+1 gruppi da 100 (es: 2001 righe, round(2001/100)=round(20,01)=20 -> perdi una riga)
      for l in range(0,(round((len(lista)/100)+0.5))):   
          id_ = lista[100*(l):100*(l+1)]
          tweet_obj = {} 
          # Metodo di ricerca tweets
          tweets = api.statuses_lookup(id_, tweet_mode = 'extended', lang = 'en')  
          for n in range(11):
            tweet_obj[n] = []
          for i in range(len(tweets)):
            tweet_obj[0].append(tweets[i].user.screen_name)
            tweet_obj[1].append(tweets[i].user.id)
            tweet_obj[2].append(tweets[i].user.verified)
            tweet_obj[3].append(tweets[i].user.created_at)
            tweet_obj[4].append(str(tweets[i].id))
            tweet_obj[5].append(pd.to_datetime(tweets[i].created_at))
            tweet_obj[6].append(tweets[i].user.description)
            tweet_obj[7].append(tweets[i].full_text)
            tweet_obj[8].append(re.findall(r'#(\w+)', tweets[i].full_text))   # estrai gli hashtags dal testo
            tweet_obj[9].append(tweets[i].source)
            tweet_obj[10].append(tweets[i].entities['urls'])

# Costruisci il dataframe con i tweets della lista di partenza
          f = pd.DataFrame(tweet_obj).rename(columns={0:'Username', 1:'User_id', 2:'User_verified', 3:'User_creation', 4:'Tweet_id', 5:'Date', 6:'User_Description', 7:'Text', 8:'Hashtags', 9:'Source', 10:'Urls'})    
          f['User_creation'] = pd.to_datetime(f['User_creation']) 
          f['User_creation'] = f['User_creation'].dt.strftime('%d/%m/%Y')  # formatta attributo 'User_creation'
          f['Date'] = pd.to_datetime(f['Date'])    # formatta attributo 'Date'
          f['Date'] = f['Date'].dt.strftime('%d/%m/%Y')
          dictio[l] = f
          
# Concatena tutti i 'l' dataframe         
      Original_post = dictio[0]
      for i in range(1,(round((len(lista)/100)+0.5))):
        Original_post = pd.concat([Original_post, dictio[i]]).sort_values(by = 'Date')

# Costruisci un nuovo indice          
      z = []
      for el in range(1,(len(Original_post)+1)):
          z.append(el)

      Original_post["Id"]=z
      Original_post.reset_index(inplace = True)
      del[Original_post['index']]
      # Elimina duplicati
      Original_post = Original_post.drop_duplicates(subset ="Tweet_id", keep = 'first')
      Original_post = Original_post.set_index('Id')
      print(len(Original_post))
      return(Original_post)   

  def join_table_function(Original_post):   # costruisci una lista con tutti i tweet_id ottenuti dai link del dataframe precedente
    a = Original_post.reset_index()    
    b = a['Urls'].to_list()
    t = a['Id'].to_list()

    h = re.compile('(https://twitter.com)/(\w+)/(status)/(\d+)')
    tweet_id = []
    for row in range(len(a)):
      c = str(b[row]).split(', ')   # dividi i vari gruppi di link
      for elem in range(len(c)):
        d = c[elem].split(': ')     # dividi i vari link
        for x in range(len(d)):
          m = h.search(d[x])      
          if m:
            tweet_id.append(m.group(4))

      
    return(tweet_id)
    
  post = Post(lista)     # la variabile post è il dataframe ottenuto dall'estrazione tweets (Post) dalla lista di tweet_id (lista)
  jt = join_table_function(post)   # la variabile jt è il dataframe ottenuto dall'estrazione tweets_id (join_table_function) dal dataframe tweets (Post(lista))

  while len(jt) != 0:
    post1 = Post(jt)
    jt = join_table_function(post1)
    if len(post1) != 0:     # evita che la mancata presenza di tweet blocchi la funzione
      if post.iloc[-1]['Tweet_id'] != post1.iloc[0]['Tweet_id']:    # se un tweet rimanda a se stesso, interrompi il ciclo for
        post = pd.concat([post, post1], ignore_index = True)
      else:
        break
    else:
      break


  post.drop_duplicates(subset = 'Tweet_id', inplace = True, keep = 'first')
  post.to_json('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON TWITTER/Final_post_{}.json'.format(n), orient='records')
  return(post, jt)


for n in diz: 
  print(n)
  if len(diz[n]) != 0:   # per ogni film
    lista = dictio_list[n]      # La lista iniziale contiene gli id estratti dagli articoli
    Prime(lista)
  